# Work Flow

## According to the listing ID (listing_ID) to match the listing reviews (use python or SQL), get the cluster category where it is located, and get three pieces of comment data according to the poly## category category (1,2,3)

## (optional) Sampling from the review data according to the proportion of the three types of clustering results to obtain three sample review data

## Filter English comment data and translate it into Chinese uniformly, keep Chinese comment data

## Combine Chinese and English comment data

## Import text analysis module

## Word segmentation topic model results

## Give business advice

# Call Baidu Translation Open Platform API module

In [19]:
# Comment data English translation module (call Baidu translation open platform API)
## Set api ID, API key
## Set conversion language
## Set the sentence to be translated quary
## Set Baidu Universal Translation API Path
## Generate a random number secret key, convert it into json format (encode function), and make URL encoding of the content (quary) after splicing.
## Create a submission format header and convert the query content into url encoding format
## Send an HTTP request and return the result in json format
## Encode Python objects into JSON strings and output the translation results in json format.


## API file name: baidu_API.py
## Import necessary libraries
import random
from hashlib import md5
import numpy as np
import pandas as pd
import urllib.request
import urllib.parse
import json
import hashlib
import time, re

## Set request conditions
appid = '20210329000750760'
appkey = 'kTb02y77pG3ZNQVW5RUG'
encoding = 'utf-8'
salt = str(random.randint(32768, 65536))
REQUEST_FAILED = -1 # Request failure code


## Define encryption function
def getMD5(content):
    m2 = hashlib.md5()
    m2.update(content.encode(encoding))
    return m2.hexdigest()

## Define the request function
def getTranslateResponce(url, data):
    data = urllib.parse.urlencode(data).encode('utf-8')
    response = urllib.request.urlopen(url, data)
    return response.read().decode('utf-8')

## Define the translation function and return the translation result
def trans(content):
    url = 'http://api.fanyi.baidu.com/api/trans/vip/translate'
    data = {}
    data['appid'] = appid
    data['salt'] = salt
    data['from'] = 'en'
    data['to'] = 'zh'
    data['q'] = content
    data['sign'] = getMD5(appid + content + salt +appkey)
    html = getTranslateResponce(url, data)
    target = json.loads(html)
    while target.get('error_code', REQUEST_FAILED) != REQUEST_FAILED:
        time.sleep(1)
        html = getTranslateResponce(url, data)
        target = json.loads(html)
    return target['trans_result'][0]['dst']

TypeError: 'str' object is not callable

# 文本分析模块

In [32]:
# Text analysis module

## Import related libraries and Chinese filter word packs (thesaurus needs to be added）
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
string=open(r'C:\Users\le\Desktop\python项目代码汇总\stopwords.txt','r',encoding='UTF-8').read()
filterwords=string.split('\n')

##Ignore warning
import logging
import warnings
logging.disable(logging.WARNING)
warnings.filterwarnings('ignore')

## Function model package
### Define the word segmentation function and store the word segmentation of each comment data in the list
def word_cut(coms):
    b=[]
    for i in jieba.cut(coms):
        if i not in filterwords:
            b.append(i)
    return b

### Numericalize text comments (eg: [1,1,1,1,0,0.....])
def get_vector(sentence,vocab):
    temp=[]
    for word in vocab:
        if word in sentence:
            temp.append(1)
        else:
            temp.append(0)
    return temp

### Package LDA model
def get_lda(params):
    corpora_words=[]
    for i in params:
        ss=word_cut(i)
        corpora_words.append(ss)
    words=[]
    for i in corpora_words:
        words+=i
    word_count=Counter(words)
    vocab=[]
    for word in word_count.keys():
        if word_count[word]>10:
            ## Word frequency parameters are adjusted according to the situation
            vocab.append(word)
    X=[]
    for se in corpora_words:
        X.append(get_vector(se,vocab))
    X=np.array(X)
    lda = LatentDirichletAllocation(n_topics=5,
                                learning_offset=50,
                                random_state=0)
   ## Build the number of topics to adjust according to the situation
    lda.fit(X)
    for i in range(5):
        index=np.argsort(lda.components_[i])[::-1]
        ##After sorting in descending order, find the corresponding index of the corresponding word under each topic on the vocab table
        print('主题',i,':',end='')
        for j in np.array(vocab)[index][:20]:
            ## According to the index, find out the top n words of each topic in descending order according to the model index
            print(j,end=' ')
        print()

# Determine whether the string contains English

In [15]:
import re

# Determine whether it contains English
str = "1!!!a"
contain_en = bool(re.search('[a-z]', str.lower()))

if contain_en:
    print("包含英文字符")
else:
    print("不包含英文字符")

包含英文字符


# Code

In [24]:
# Get the translated comment data and save it in the airbnb_Selection_comment table

df=pd.read_excel(r'C:\Users\le\Desktop\Business Data Analysis\爱彼迎英文评论测试.xlsx')
row = df.shape[0]  # 获得行数
col = df.shape[1]  # 获得列数
original_list = []
translation_list =[]
from baidu_API import *
## Extract all comment text, save the text before and after translation in a different list, and save it as excel
for c in range(1,col):
    for r in range(0,row):
        comment = df.iloc[r][c]
        original_list.append(comment)
        translation = str(trans(str(commentP).replace('\n','')))  
        translation_list.append(translation)
        translation_sheet=pd.DataFrame({'原文':original_list,'译文':translation_list})
        translation_sheet.to_csv('C:/Users/le/Desktop/Business Data Analysis/airbnb_Translation_comment.csv',encoding = 'utf-8')

TypeError: 'str' object is not callable

In [2]:
# Import the first and third types of sampled review data and the second type of overall review data
# Adjust model parameters
# Analysis results

In [70]:
# Model module
# Text analysis module

## Import related libraries and Chinese filter word packs (thesaurus needs to be added)
import jieba
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from collections import Counter
string=open(r'C:\Users\le\Desktop\python项目代码汇总\stopwords.txt','r',encoding='UTF-8').read()
punctuation='[’!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~\n。！，“”]+ ｡ ､ ｡; …  ｡'
filterwords=string.split('\n')

##Ignore warning
import logging
import warnings
logging.disable(logging.WARNING)
warnings.filterwarnings('ignore')

## Function model package
### Define the word segmentation function and store the word segmentation of each comment data in the list
def word_cut(coms):
    b=[]
    for i in jieba.cut(coms):
        if i not in filterwords:
            i=re.sub(punctuation,'',i) ## 过滤标点
            i=re.sub('\n','',i)
            i=re.sub('[\d]','',i)## 过滤数字
            b.append(i)
    return b

### Numericalize text comments (eg: [1,1,1,1,0,0.....])
def get_vector(sentence,vocab):
    temp=[]
    for word in vocab:
        if word in sentence:
            temp.append(1)
        else:
            temp.append(0)
    return temp


### 封装LDA模型
def get_lda(params):
    corpora_words=[]
    for i in params:
        ss=word_cut(i)
        corpora_words.append(ss)
    words=[]
    for i in corpora_words:
        words+=i
    word_count=Counter(words)
    vocab=[]
    for word in word_count.keys():
        if word_count[word]>100:
            ## Word frequency parameters are adjusted according to the situation
            vocab.append(word)
    X=[]
    for se in corpora_words:
        X.append(get_vector(se,vocab))
    X=np.array(X)
    lda = LatentDirichletAllocation(n_topics=5,
                                learning_offset=50,
                                random_state=0)
  ## Build the number of topics to adjust according to the situation
    lda.fit(X)
    for i in range(5):
        index=np.argsort(lda.components_[i])[::-1]
        ##After sorting in descending order, find the corresponding index of the corresponding word under each topic on the vocab table
        print('主题',i,':',end='')
        for j in np.array(vocab)[index][:20]:
            ## According to the index, find the top n words of each topic in descending order according to the model index
            print(j,end=' ')
        print()

# 第一类（质量好的优质房源评论）-----超赞房东

In [50]:
df2=pd.read_csv(r'C:/Users/le/Desktop/Business Data Analysis/segment1_sample.csv',encoding='utf-8') # 共计3000条

In [74]:
list_comment=df2['comments']
get_lda(list_comment)

主题 0 :干净 交通 ｡ 整洁 便利 齐全 设施 位置 环境 安静 温馨 满意 适合 热情 舒适 小区 推荐 住宿 装修 ~ 
主题 1 :挺 房主 阿姨 棒 地铁 可爱 体验 吃 热情  楼下 生活 ~ 很近 家 东西 推荐 晚上 贴心 设施 
主题 2 :｡ 地铁站  位置 近 回复 干净 地理位置 很近 距离 步行 小区 入住 出行 地铁 一个 走 分钟 热情 装修 
主题 3 :｡ 体验 入住 北京 性价比 住 下次 高 推荐 真的 舒适 民宿 感觉 干净 胡同 位置 很多 热情 ~ 还会 
主题 4 :舒服 住 特别 ｡ 超级 小姐姐 ~ 温馨 地方 热情 干净 下次 家里 床 真的 北京 开心 可爱 感觉 近 


# 第三类（质量好的房源评论）-----非超赞房东

In [54]:
df3=pd.read_csv(r'C:/Users/le/Desktop/Business Data Analysis/segment3_sample.csv',encoding='utf-8')  # 共计1500条

In [73]:
list_comment=df3['comments']
get_lda(list_comment)
# 参数说明：主题数：5 每个主题显示20个词   高频词定义标准：累计出现100次

# 结论分析：
# 1 入住体验类：年轻女性类的房东的的亲和力较高，给租客传达出了一种温馨热情的入住体验
# 2 地理位置类：距离地铁站距离近，地理位置优越的房子，提供了更为便利的交通
# 3 房屋配置类：设施配置齐全，房屋环境干净，令租客感觉性价比高

# Business Insight：
# 1  筛选优质评论中含有关键词的评论，匹配其对应的优质房源，撰写优质干净房源文案，以滑动banner位的形式上传到APP首页，
# 给平台上北京地区的其他房东提供改善房源的方向。

主题 0 :｡ 热情 推荐 设施 地理位置 特别 齐全 干净 温馨 性价比 很多 位置 整洁 入住 ~ 挺 下次 ､ 北京 住 
主题 1 :位置 ｡  小区 挺 一个 地铁站 干净 性价比 住 ､ 很多 ~ 整洁 入住 齐全 体验 设施 北京 热情 
主题 2 :｡ 住 舒服 北京 下次 近 感觉 干净 特别 地铁站 热情 很多  温馨 ､ 性价比 挺 整洁 位置 ~ 
主题 3 :交通 便利 ｡ 干净 热情 地铁站 位置 齐全 е о а н т 近 北京 整洁 温馨 下次 住 小区 
主题 4 :体验 入住 ｡ 干净 ~ 舒适 整洁 热情 住 温馨 位置 性价比 下次 地铁站 挺 近 齐全  设施 舒服 


# 第二类（质量差的房源评论）

In [7]:
df1=pd.read_csv(r'C:/Users/le/Desktop/Business Data Analysis/segment2.csv',encoding='utf-8')  # 共计584条

In [47]:
list_comment=df1['comments']
get_lda(list_comment)
# 参数说明：主题数：5 每个主题显示20个词   高频词定义标准：累计出现10次

# 结论分析：
# 1 取消预订类： 房东在租客临近入住第1,2,3天时，由于房东取消订单导致系统自动发布不良评价
# 2 硬件设施类：租客反应厨房和卫生间里得硬件因为清洁问题影响用户的入住体验

# Business Insight：
# 1 房东侧：在接受订单时设置惩罚提醒 。
# 2 在房东平均分计算体系中加入厨房和卫生间的评级维度，关注厨房和卫生间的权重比例。
# 3 将有厨房和卫生间有负面反馈的房东发送短信push，促使其通过改善这两者的卫生条件来提升其平均分。

主题 0 :自动 主人 取消 预订 这是 投递 天 抵达 到达  前一天 前三天 前两天 估计 ｡ 设施 ; 位置 数据 点 
主题 1 :前两天 到达 这是 主人 投递 取消 自动 预订 前一天 一个 照片 地方 电话 回复 晚上 找 很多 房源 走 干净 
主题 2 :投递 取消 预订 这是 自动 主人 天  到达 抵达 房源 估计 ｡ 地方 购买 设施 住 前一天 地区 没 
主题 3 :｡  入住 地方 一个 干净 体验 卫生间 没 住 差 房源 找 真的 晚上 空调 卫生 厨房 设施 发现 
主题 4 :住 ｡ 图片 没 睡 不好 一个 床 房主 回复 电话 晚上 房源 点 走 真的 取消 坏 两个 差 
